In [1]:
!pip install faiss-cpu sentence-transformers transformers

In [2]:
documents = [
    "RAG significa Retrieval Augmented Generation, uma técnica que combina recuperação de documentos com modelos de linguagem.",
    "FAISS é uma biblioteca do Facebook para busca vetorial em grandes coleções de embeddings.",
    "LLMs podem usar RAG para acessar informações externas sem precisar memorizar tudo nos pesos do modelo.",
    "HuggingFace fornece modelos de embeddings e modelos generativos para implementar RAG."
]


In [3]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeddings = embedder.encode(documents)
embeddings[:2]  # para ver um preview


array([[-7.86589310e-02, -1.13060093e-02, -2.25555785e-02,
         7.38768280e-03, -9.56714302e-02,  6.18261062e-02,
         1.56401861e-02,  4.37650131e-04,  5.75030260e-02,
        -2.17895713e-02,  9.02783498e-02,  6.19908236e-02,
         5.69356568e-02, -5.18538617e-02,  9.52456333e-03,
        -7.61134550e-03,  2.71676370e-04,  9.04104784e-02,
        -2.10468825e-02, -5.49692959e-02,  6.76912814e-02,
         9.84912217e-02,  5.96747959e-05, -3.52148190e-02,
        -3.92156048e-03,  3.44415419e-02, -8.90657231e-02,
        -1.38732158e-02,  7.48324469e-02, -2.76185255e-02,
         1.94502976e-02,  1.50965422e-01,  1.66499987e-02,
         7.36228451e-02, -3.50925587e-02, -1.95667092e-02,
        -6.99788481e-02,  1.74974613e-02,  3.69148515e-02,
         5.76143414e-02, -1.26711383e-01, -9.04475991e-03,
        -1.42752593e-02, -1.46887302e-02,  9.26230550e-02,
        -5.22955321e-02, -1.15238860e-01,  5.24035878e-02,
         2.87715858e-03,  6.94967480e-03, -5.40153682e-0

In [5]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


In [6]:
def rag_query(query, top_k=2):
    query_emb = embedder.encode([query])
    distances, indices = index.search(np.array(query_emb), top_k)

    retrieved_docs = [documents[i] for i in indices[0]]
    context = "\n".join(retrieved_docs)

    return context


In [7]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
def rag_answer(query):
    context = rag_query(query)

    prompt = f"Use as informações abaixo para responder:\n\n{context}\n\nPergunta: {query}"

    answer = generator(prompt, max_length=150, do_sample=False)
    return answer[0]['generated_text'], context


In [9]:
query = "Para que serve o RAG?"
answer, retrieved = rag_answer(query)

print("📌 Pergunta:", query)
print("\n📚 Documentos Recuperados:\n", retrieved)
print("\n🤖 Resposta do Modelo:\n", answer)


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📌 Pergunta: Para que serve o RAG?

📚 Documentos Recuperados:
 LLMs podem usar RAG para acessar informações externas sem precisar memorizar tudo nos pesos do modelo.
FAISS é uma biblioteca do Facebook para busca vetorial em grandes coleções de embeddings.

🤖 Resposta do Modelo:
 LLMs can use RAG to acessar external information without precisar memorizar all nos pesos do model. FAISS is a biblioteca do Facebook para busca vetorial em grande coleçes de embeddings.


In [10]:
print('Enzo Bognar')

Enzo Bognar
